## Código desenvolvido em Java para criação da aplicação que utilize a base de dados compras.txt e lista a soma de cada produto vendido por cidade. Projeto inicial chamado somacidadeproduto contendo a estrutura-base das classes

### 1-Desenvolvimento das 3 classes Mapper, Reducer e Driver para implementação da aplicação

In [ ]:
// Desenvolvimento da classe Driver SomaCidadeProdutoDriver.java

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.FloatWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class SomaCidadeProdutoDriver {

	public static void main(String[] args) throws Exception {

		Configuration conf = new Configuration();
		Job job = Job.getInstance(conf, "somacidadeprodutodriver");
		job.setJarByClass(SomaCidadeProdutoDriver.class);
		job.setMapperClass(SomaCidadeProdutoMapper.class);
		job.setMapOutputKeyClass(Text.class);
		job.setMapOutputValueClass(FloatWritable.class);
		job.setReducerClass(SomaCidadeProdutoReducer.class);
		job.setOutputKeyClass(Text.class);
		job.setOutputValueClass(FloatWritable.class);
		FileInputFormat.addInputPath(job, new Path(args[0]));
		FileOutputFormat.setOutputPath(job, new Path(args[1]));
		System.exit(job.waitForCompletion(true) ? 0 : 1);

	}
}

In [ ]:
// Desenvolvimento da classe Mapper SomaCidadeProdutoMapper.java

import java.io.IOException;
import org.apache.hadoop.io.FloatWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

public class SomaCidadeProdutoMapper extends Mapper<Object, Text, Text, FloatWritable> {

	private final static FloatWritable valor = new FloatWritable(0);
	private final static Text cidadeproduto = new Text();

	public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
		
		String[] linha=value.toString().split(";");		
		cidadeproduto.set(linha[2] +" "+linha[3]);
		valor.set(Float.parseFloat(linha[4].replaceAll(":", ".")));	
		context.write(cidadeproduto, valor);		
	}
}

In [ ]:
// Desesenvolvimento da classe Reducer SomaCidadeProdutoReducer.java

import java.io.IOException;
import org.apache.hadoop.io.FloatWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;

public class SomaCidadeProdutoReducer extends	Reducer<Text, FloatWritable, Text, FloatWritable> {
	
	public void reduce(Text key, Iterable<FloatWritable> values, Context context) throws IOException, InterruptedException {
		long sumValue = 0;
		
		for (FloatWritable value : values) {
			sumValue += value.get();
		}
		context.write(key, new FloatWritable(sumValue));
	}
}

### 2-Geração do arquivo JAR da aplicação e definição do path:

Exemplo: /home/cloudera/Documents/somacidadeproduto.jar

### 3-Execução do job JAR no ambiente Hadoop e consumo do arquivo bases/compras.txt já armazenado no HDFS:

hadoop jar ~/Documents/somacidadeproduto.jar SomaCidadeProdutoDriver bases/compras.txt saida/cidade_produto_soma